In [14]:
import numpy as np
import pandas as pd

In [30]:
pd.read_csv('Truck-Crash-Data/origPositions.csv')

,NodeNumber,xPos,yPos,zPos
0,2023483,4092.1599,323.10001,487.19998
1,2023484,4079.5200,324.01999,493.63998
2,2023485,4066.8899,324.95999,500.07999
3,2023486,4054.2600,325.89999,506.51999
4,2023487,4041.6299,326.85999,512.95996
...,...,...,...,...
3228,2028447,3497.6399,-402.69998,399.26999
3229,2028448,3491.9600,-427.53998,396.35999
3230,2028449,3502.1399,-423.73999,400.56000
3231,2028450,3474.6899,-407.62000,391.42999


We can ignore the node numbers, they're always in order

In [67]:
origPositions = np.array(pd.read_csv('Truck-Crash-Data/origPositions.csv'))[:,1:]
displacementSim = np.zeros((126, origPositions.shape[0], 3))
for i in range(126):
    displacementSim[i] = np.array(pd.read_csv('Truck-Crash-Data/DisplacementSimulation' + str(i+1) + '.csv'))[:,1:]
displacementSim.shape

(126, 3233, 3)